In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import VisualBertModel, VisualBertConfig, BertTokenizer, VisualBertForQuestionAnswering
from sklearn.model_selection import train_test_split
import pickle
import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
class Cifar100Dataset(Dataset):
    def __init__(self, visual_embeddings, labels, tokenizer):
        self.visual_embeddings = visual_embeddings
        self.labels = labels
        self.tokenizer = tokenizer
        self.num_labels = len(set(self.labels))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        visual_embedding = self.visual_embeddings[idx]
        # text_inputs = tokenizer(text_inputs, padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        text_inputs = tokenizer("", padding="max_length", max_length=32, truncation=True, return_tensors="pt")
        
        # remove batch dimension
        for k,v in text_inputs.items():
            text_inputs[k] = v.squeeze()
            
        label = torch.zeros(self.num_labels)
        label[self.labels[idx]] = 1
        return visual_embedding, text_inputs, label

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [19]:
class VisualBERTForImageClassification(nn.Module):
    def __init__(self, model_name, config):
        super().__init__()
        self.visual_bert = VisualBertForQuestionAnswering.from_pretrained(model_name, config=config)
        # self.projection = nn.Linear(256*7*7, visual_embedding_size * 256)
        # self.classifier = nn.Linear(config.hidden_size, num_classes)
        # self.classifier = nn.Sequential(
        #     nn.Linear(in_features=self.visual_bert.config.hidden_size, out_features=1536, bias=True),
        #     nn.LayerNorm((1536,), eps=1e-05, elementwise_affine=True),
        #     nn.GELU(approximate='none'),
        #     nn.Linear(in_features=1536, out_features=num_classes, bias=True)
        # )

    def forward(self, visual_embeddings, text_inputs):
#         # Project to the size that VisualBert expects
#         input_visual_embeddings = self.projection(visual_embeddings.view(visual_embeddings.size(0), -1))
        
#         # Reshape the visual embeddings back to 3D tensor
#         input_visual_embeddings = input_visual_embeddings.view(input_visual_embeddings.size(0), 256, -1)
                
        outputs = self.visual_bert(
            input_ids=text_inputs["input_ids"],
            attention_mask=text_inputs["attention_mask"],
            token_type_ids=text_inputs["token_type_ids"],
            visual_embeds=visual_embeddings
        )
        return outputs

In [5]:
def collate_fn(batch):
        visual_embeddings, text_inputs, labels = zip(*batch)
        
        # Flatten the visual embeddings
        visual_embeddings = [embedding.view(embedding.size(0), -1) for embedding in visual_embeddings]
        
        visual_embeddings = torch.stack(visual_embeddings)
        
        labels = torch.stack(labels)
        
        input_ids = torch.stack([item['input_ids'] for item in text_inputs])
        attention_mask = torch.stack([item['attention_mask'] for item in text_inputs])
        token_type_ids = torch.stack([item['token_type_ids'] for item in text_inputs])
        
        text_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids
        }
        
                
        return visual_embeddings, text_inputs, labels

In [6]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for visual_embeddings, text_inputs, labels in dataloader:
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(visual_embeddings, text_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

In [7]:
with open('../cifar100-train-embeddings.pkl', 'rb') as f:
    train_embeddings_all = pickle.load(f)

with open('../cifar100-test-embeddings.pkl', 'rb') as f:
    test_embeddings_all = pickle.load(f)

In [8]:
def curtail_embeddings(embeddings):
    lowest = 1000
    
    for e in embeddings['embeddings']:
        lowest = min(lowest, e.size(0))
    
    for idx, e in enumerate(embeddings['embeddings']):
        embeddings['embeddings'][idx] = e[:lowest]

In [9]:
curtail_embeddings(train_embeddings_all)
curtail_embeddings(test_embeddings_all)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_embeddings = train_embeddings_all['embeddings']
train_labels = train_embeddings_all['fine_labels']

val_embeddings = test_embeddings_all['embeddings']
val_labels = test_embeddings_all['fine_labels']
num_classes = len(set(train_labels))

In [20]:
config = VisualBertConfig.from_pretrained("uclanlp/visualbert-vqa")
config.num_labels = 100
config.visual_embedding_dim = 1024

model = VisualBERTForImageClassification("uclanlp/visualbert-nlvr2-coco-pre", config=config)
model = model.to(device)

Some weights of the model checkpoint at uclanlp/visualbert-nlvr2-coco-pre were not used when initializing VisualBertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing VisualBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VisualBertForQuestionAnsweri

In [21]:
model

VisualBERTForImageClassification(
  (visual_bert): VisualBertForQuestionAnswering(
    (visual_bert): VisualBertModel(
      (embeddings): VisualBertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=1)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (visual_token_type_embeddings): Embedding(2, 768)
        (visual_position_embeddings): Embedding(512, 768)
        (visual_projection): Linear(in_features=1024, out_features=768, bias=True)
      )
      (encoder): VisualBertEncoder(
        (layer): ModuleList(
          (0): VisualBertLayer(
            (attention): VisualBertAttention(
              (self): VisualBertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True

In [22]:
train_dataset = Cifar100Dataset(train_embeddings, train_labels, tokenizer)
val_dataset = Cifar100Dataset(val_embeddings, val_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)

In [23]:
for name, param in model.named_parameters():
    if 'cls' in name or 'pooler' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.25)

In [28]:
best_params = None
best_val_accuracy = -1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    train_predictions = []
    train_labels_eval = []
    
    for visual_embeddings, text_inputs, labels in tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(visual_embeddings, text_inputs).logits
        _, preds = torch.max(outputs, 1)
        
        train_predictions.extend(preds.cpu().numpy())
        train_labels_eval.extend(labels.cpu().numpy())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()
    
    train_loss = total_loss / len(train_dataloader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_predictions = []
    val_labels_eval = []
        
    with torch.no_grad():
        for visual_embeddings, text_inputs, labels in val_dataloader:
            visual_embeddings = visual_embeddings.to(device)
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            labels = labels.to(device)
            
            outputs = model(visual_embeddings, text_inputs).logits
            _, preds = torch.max(outputs, 1)
            
            val_predictions.extend(preds.cpu().numpy())
            val_labels_eval.extend(labels.cpu().numpy())
    
    val_labels_idx = [np.argmax(tensor) for tensor in val_labels_eval]
    val_accuracy = accuracy_score(val_labels_idx, val_predictions)
    
    train_labels_idx = [np.argmax(tensor) for tensor in train_labels_eval]
    train_accuracy = accuracy_score(train_labels_idx, train_predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/15: 100%|██████████| 196/196 [01:02<00:00,  3.14it/s]


Epoch 1/15, Loss: 4.6125, Training Acc: 0.0121, Validation Accuracy: 0.0376


Epoch 2/15: 100%|██████████| 196/196 [01:02<00:00,  3.15it/s]


Epoch 2/15, Loss: 4.5862, Training Acc: 0.0195, Validation Accuracy: 0.0583


Epoch 3/15: 100%|██████████| 196/196 [01:02<00:00,  3.15it/s]


...

Epoch 13/15, Loss: 4.4298, Training Acc: 0.0962, Validation Accuracy: 0.1308


Epoch 14/15: 100%|██████████| 196/196 [00:53<00:00,  3.70it/s]


Epoch 14/15, Loss: 4.4279, Training Acc: 0.0960, Validation Accuracy: 0.1317


Epoch 15/15: 100%|██████████| 196/196 [00:53<00:00,  3.69it/s]


Epoch 15/15, Loss: 4.4239, Training Acc: 0.0988, Validation Accuracy: 0.1310


Try with pooler still frozen

In [29]:
model = VisualBERTForImageClassification("uclanlp/visualbert-nlvr2-coco-pre", config=config)
model = model.to(device)

train_dataset = Cifar100Dataset(train_embeddings, train_labels, tokenizer)
val_dataset = Cifar100Dataset(val_embeddings, val_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, collate_fn=collate_fn)

for name, param in model.named_parameters():
    if 'cls' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.25)


best_params = None
best_val_accuracy = -1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    train_predictions = []
    train_labels_eval = []
    
    for visual_embeddings, text_inputs, labels in tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        visual_embeddings = visual_embeddings.to(device)
        text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(visual_embeddings, text_inputs).logits
        _, preds = torch.max(outputs, 1)
        
        train_predictions.extend(preds.cpu().numpy())
        train_labels_eval.extend(labels.cpu().numpy())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()
    
    train_loss = total_loss / len(train_dataloader)
    
    # Evaluate the model on the validation set
    model.eval()
    val_predictions = []
    val_labels_eval = []
        
    with torch.no_grad():
        for visual_embeddings, text_inputs, labels in val_dataloader:
            visual_embeddings = visual_embeddings.to(device)
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}
            labels = labels.to(device)
            
            outputs = model(visual_embeddings, text_inputs).logits
            _, preds = torch.max(outputs, 1)
            
            val_predictions.extend(preds.cpu().numpy())
            val_labels_eval.extend(labels.cpu().numpy())
    
    val_labels_idx = [np.argmax(tensor) for tensor in val_labels_eval]
    val_accuracy = accuracy_score(val_labels_idx, val_predictions)
    
    train_labels_idx = [np.argmax(tensor) for tensor in train_labels_eval]
    train_accuracy = accuracy_score(train_labels_idx, train_predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Training Acc: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Some weights of the model checkpoint at uclanlp/visualbert-nlvr2-coco-pre were not used when initializing VisualBertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing VisualBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VisualBertForQuestionAnsweri

Epoch 1/15, Loss: 4.6070, Training Acc: 0.0136, Validation Accuracy: 0.0288


Epoch 2/15: 100%|██████████| 391/391 [01:01<00:00,  6.40it/s]


Epoch 2/15, Loss: 4.5730, Training Acc: 0.0255, Validation Accuracy: 0.0730


Epoch 3/15: 100%|██████████| 391/391 [01:03<00:00,  6.19it/s]


...

Epoch 10/15, Loss: 4.3857, Training Acc: 0.0999, Validation Accuracy: 0.1210


Epoch 11/15: 100%|██████████| 391/391 [00:56<00:00,  6.96it/s]


Epoch 11/15, Loss: 4.3670, Training Acc: 0.1046, Validation Accuracy: 0.1242


Epoch 12/15: 100%|██████████| 391/391 [01:36<00:00,  4.04it/s]


KeyboardInterrupt: 